In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
import pylab as py
import scipy.stats as stats 
from scipy.stats import norm, skew, probplot
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.decomposition import PCA

In [2]:
from Feat_eng import Y_train, X_train, Y_test, X_test, c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test  

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().dro

## Run a Lasso Regression

In [3]:
# Run a Lasso Regression on all variables

scaler = StandardScaler().fit(X_train)
features = scaler.transform(X_train)
X_std = pd.DataFrame(features, columns = X_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(X_std, Y_train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(X_std,Y_train)

print('Score for training data:', las.score(X_std,Y_train))

pred = las.predict(X_std)
pred_unlog = np.exp(pred)
Y_unlog = np.exp(Y_train)
train_RMSE = mean_squared_error(Y_unlog, pred_unlog, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(X_test)
X_test_std = pd.DataFrame(test_features, columns = X_test.columns)

print('Score for test data:', las.score(X_test_std,Y_test))

pred_test = las.predict(X_test_std)
pred_test_unlog = np.exp(pred_test)
Y_test_unlog = np.exp(Y_test)
Test_RMSE = mean_squared_error(Y_test_unlog, pred_test_unlog, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = X_test_std.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9879606079655315
(not log) RMSE for training data is: 76.62356976513438
Score for test data: 0.9869111709288596
(not log) RMSE for test data is: 83.5136289773991


,Feature,Importance,Abs Importance,ranking
128,log_value,0.409726,0.409726,1
126,interest,-0.008158,0.008158,2
40,owner_occupied_housing_units_lower_value_quartile,0.003127,0.003127,3
125,gdp,0.002229,0.002229,4
21,vacant_housing_units,-0.000556,0.000556,5
...,...,...,...,...
41,owner_occupied_housing_units_median_value,0.000000,0.000000,133
39,renter_occupied_housing_units_paying_cash_medi...,0.000000,0.000000,134
38,aggregate_travel_time_to_work,0.000000,0.000000,135
37,commute_45_59_mins,0.000000,0.000000,136


In [4]:
r2_train = las.score(X_std, Y_train)
r2_test = las.score(X_test_std, Y_test)
lasso_output = pd.DataFrame()
lasso_output['all'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
lasso_output

,all
RMSE_train,76.623570
RMSE_test,83.513629
R2_train,0.987961
R2_test,0.986911


In [5]:
import pickle
Pkl_Filename = "Pickle_lasso_all.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(las, file)

## Run a PCA 

In [6]:
scaler_pca = StandardScaler().fit(X_train)
features = scaler_pca.transform(X_train)
df_scal = pd.DataFrame(features, columns = X_train.columns)

In [7]:
pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df_scal)

print(df_reduced.shape)

df_reduced = pd.DataFrame(pca.components_, columns = X_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))


(5122, 35)


In [8]:
# Try to reduce features by finding most important one for each of the dimensions
feat = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat.append(ff)

In [9]:
feat.append('log_value')
feat

['total_pop',
 'income_per_capita',
 'dwellings_1_units_detached',
 'vacant_housing_units_for_rent',
 'hispanic_ratio',
 'cpi',
 'in_undergrad_college',
 'white_ratio',
 'median_age',
 'median_year_structure_built',
 'asian_including_hispanic',
 'housing_built_2005_or_later',
 'dwellings_1_units_attached',
 'vacant_housing_units',
 'gdp',
 'interest',
 'amerindian_including_hispanic',
 'male_ratio',
 'gini_index',
 'dwellings_10_to_19_units',
 'employed_public_administration',
 'est',
 'commute_20_24_mins',
 'commute_5_9_mins',
 'percent_income_spent_on_rent',
 'housing_built_2000_to_2004',
 'renter_occupied_housing_units_paying_cash_median_gross_rent',
 'different_house_year_ago_same_city',
 'dwellings_5_to_9_units',
 'different_house_year_ago_different_city',
 'housing_built_1939_or_earlier',
 'dwellings_3_to_4_units',
 'employed_construction',
 'Cluster',
 'employed_information',
 'log_value']

In [10]:
X_red_train = X_train[feat]
X_red_test = X_test[feat]

In [11]:
# Run Lasso Regression on PCA variables:
scaler_red = StandardScaler().fit(X_red_train)
features_red = scaler_red.transform(X_red_train)
X_red_std = pd.DataFrame(features_red, columns = X_red_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv_red = LassoCV(alphas = alphas, random_state=0)
lassocv_red.fit(X_red_std, Y_train)
lassocv_red_alpha = lassocv_red.alpha_

las_red = Lasso(alpha = lassocv_red.alpha_, random_state = 0)
las_red.fit(X_red_std,Y_train)

print('Score for training data:', las_red.score(X_red_std,Y_train))

pred_red = las_red.predict(X_red_std)
pred_red = np.exp(pred_red)
Y_train_red = np.exp(Y_train)
train_RMSE_red = mean_squared_error(Y_train_red, pred_red, squared = False)
print('(not log) RMSE for training data is:', train_RMSE_red)

test_features_red = scaler_red.transform(X_red_test)
X_red_test_std = pd.DataFrame(test_features_red, columns = X_red_test.columns)

print('Score for test data:', las_red.score(X_red_test_std,Y_test))

pred_test_red = las_red.predict(X_red_test_std)
pred_test_red = np.exp(pred_test_red)
Y_test_red = np.exp(Y_test)
Test_RMSE_red = mean_squared_error(Y_test_red, pred_test_red, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE_red)

las_imp_red = pd.DataFrame()

las_imp_red['Feature'] = X_red_test_std.columns
las_imp_red['Importance'] = list(las_red.coef_)
las_imp_red['Abs Importance'] = abs(las_imp_red['Importance'])
las_imp_red = las_imp_red.sort_values(by = 'Abs Importance', ascending = False)
las_imp_red

ranking_red = list(range(1,las_imp_red.shape[0]+1))
las_imp_red['ranking'] = ranking_red
las_imp_red

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9879161855379601
(not log) RMSE for training data is: 76.57667875044893
Score for test data: 0.9862853599303031
(not log) RMSE for test data is: 86.46024920058032


,Feature,Importance,Abs Importance,ranking
35,log_value,0.412199,0.412199,1
15,interest,-0.008393,0.008393,2
14,gdp,0.002033,0.002033,3
13,vacant_housing_units,-0.000747,0.000747,4
26,renter_occupied_housing_units_paying_cash_medi...,0.000000,0.000000,5
22,commute_20_24_mins,0.000000,0.000000,6
23,commute_5_9_mins,-0.000000,0.000000,7
24,percent_income_spent_on_rent,-0.000000,0.000000,8
25,housing_built_2000_to_2004,0.000000,0.000000,9
27,different_house_year_ago_same_city,-0.000000,0.000000,10


In [12]:
r2_train = las_red.score(X_red_std,Y_train)
r2_test = las_red.score(X_red_test_std, Y_test)
lasso_output['all_PCA'] = [train_RMSE_red, Test_RMSE_red, r2_train, r2_test]
lasso_output

,all,all_PCA
RMSE_train,76.623570,76.576679
RMSE_test,83.513629,86.460249
R2_train,0.987961,0.987916
R2_test,0.986911,0.986285


In [13]:
Pkl_Filename = "Pickle_lasso_all_PCA.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(las, file)

## Run a Lasso Regression on the Clusters

In [14]:
# Run Lasso Regression on Cluster 1:
scaler = StandardScaler().fit(c1_train)
features = scaler.transform(c1_train)
c1_strain = pd.DataFrame(features, columns = c1_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c1_strain, Y_c1train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c1_strain,Y_c1train)

print('Score for training data:', las.score(c1_strain,Y_c1train))

pred = las.predict(c1_strain)
pred = np.exp(pred)
Y_c1_train = np.exp(Y_c1train)
train_RMSE = mean_squared_error(Y_c1_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c1_test)
c1_stest = pd.DataFrame(test_features, columns = c1_test.columns)

print('Score for test data:', las.score(c1_stest,Y_c1test))

pred_test = las.predict(c1_stest)
pred_test = np.exp(pred_test)
Y_c1_test = np.exp(Y_c1test)
Test_RMSE = mean_squared_error(Y_c1_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c1_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9856491625045858
(not log) RMSE for training data is: 90.97670557020622
Score for test data: 0.9657896964172876
(not log) RMSE for test data is: 139.8050728766678


,Feature,Importance,Abs Importance,ranking
128,log_value,0.337524,0.337524,1
126,interest,-0.012287,0.012287,2
40,owner_occupied_housing_units_lower_value_quartile,0.003510,0.003510,3
56,housing_built_1939_or_earlier,0.003448,0.003448,4
85,employed_public_administration,-0.002907,0.002907,5
...,...,...,...,...
39,renter_occupied_housing_units_paying_cash_medi...,0.000000,0.000000,133
38,aggregate_travel_time_to_work,-0.000000,0.000000,134
37,commute_45_59_mins,-0.000000,0.000000,135
36,commute_30_34_mins,-0.000000,0.000000,136


In [15]:
r2_train = las.score(c1_strain,Y_c1train)
r2_test = las.score(c1_stest,Y_c1test)
lasso_output['Clust_1'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1
RMSE_train,76.623570,76.576679,90.976706
RMSE_test,83.513629,86.460249,139.805073
R2_train,0.987961,0.987916,0.985649
R2_test,0.986911,0.986285,0.965790


In [16]:
Pkl_Filename = "Pickle_lasso_c1.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(las, file)

In [17]:
# Run a PCA on Cluster 1:
scaler_pca = StandardScaler().fit(c1_train)
features = scaler_pca.transform(c1_train)
df1_scal = pd.DataFrame(features, columns = c1_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df1_scal)

df_reduced = pd.DataFrame(pca.components_, columns = c1_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat1 = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat1.append(ff)

feat1.append('log_value')

c1_train = c1_train[feat1]
c1_test = c1_test[feat1]

# Run Lasso Regression on Cluster 1 with PCA variables:

scaler = StandardScaler().fit(c1_train)
features = scaler.transform(c1_train)
c1_strain = pd.DataFrame(features, columns = c1_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c1_strain, Y_c1train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c1_strain,Y_c1train)

print('Score for training data:', las.score(c1_strain,Y_c1train))

pred = las.predict(c1_strain)
pred = np.exp(pred)
Y_c1_train = np.exp(Y_c1train)
train_RMSE = mean_squared_error(Y_c1_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c1_test)
c1_stest = pd.DataFrame(test_features, columns = c1_test.columns)

print('Score for test data:', las.score(c1_stest,Y_c1test))

pred_test = las.predict(c1_stest)
pred_test = np.exp(pred_test)
Y_c1_test = np.exp(Y_c1test)
Test_RMSE = mean_squared_error(Y_c1_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c1_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9853525204768286
(not log) RMSE for training data is: 92.07859134638352
Score for test data: 0.9873485870404418
(not log) RMSE for test data is: 79.4622792385401


,Feature,Importance,Abs Importance,ranking
33,log_value,0.340296,0.340296,1
6,cpi,-0.011508,0.011508,2
5,employed_public_administration,-0.003197,0.003197,3
23,housing_built_1939_or_earlier,0.002836,0.002836,4
14,gdp,0.001774,0.001774,5
10,asian_ratio,0.000812,0.000812,6
26,housing_built_2000_to_2004,-0.000000,0.000000,7
21,est,0.000000,0.000000,8
22,dwellings_3_to_4_units,0.000000,0.000000,9
24,in_undergrad_college,-0.000000,0.000000,10


In [18]:
r2_train = las.score(c1_strain,Y_c1train)
r2_test = las.score(c1_stest,Y_c1test)
lasso_output['Clust_1_PCA'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA
RMSE_train,76.623570,76.576679,90.976706,92.078591
RMSE_test,83.513629,86.460249,139.805073,79.462279
R2_train,0.987961,0.987916,0.985649,0.985353
R2_test,0.986911,0.986285,0.965790,0.987349


In [19]:
Pkl_Filename = "Pickle_lasso_c1_PCA.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(las, file)

In [20]:
# Run Lasso Regression on Cluster 2:

scaler = StandardScaler().fit(c2_train)
features = scaler.transform(c2_train)
c2_strain = pd.DataFrame(features, columns = c2_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c2_strain, Y_c2train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c2_strain,Y_c2train)

print('Score for training data:', las.score(c2_strain,Y_c2train))

pred = las.predict(c2_strain)
pred = np.exp(pred)
Y_c2_train = np.exp(Y_c2train)
train_RMSE = mean_squared_error(Y_c2_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c2_test)
c2_stest = pd.DataFrame(test_features, columns = c2_test.columns)

print('Score for test data:', las.score(c2_stest,Y_c2test))

pred_test = las.predict(c2_stest)
pred_test = np.exp(pred_test)
Y_c2_test = np.exp(Y_c2test)
Test_RMSE = mean_squared_error(Y_c2_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c2_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9851272671720659
(not log) RMSE for training data is: 67.52056582700475
Score for test data: 0.9887063552312815
(not log) RMSE for test data is: 64.86491535359875


,Feature,Importance,Abs Importance,ranking
128,log_value,0.376026,0.376026,1
126,interest,-0.005451,0.005451,2
40,owner_occupied_housing_units_lower_value_quartile,0.004532,0.004532,3
23,median_rent,0.004464,0.004464,4
125,gdp,0.003755,0.003755,5
...,...,...,...,...
42,owner_occupied_housing_units_upper_value_quartile,0.000000,0.000000,133
41,owner_occupied_housing_units_median_value,0.000000,0.000000,134
39,renter_occupied_housing_units_paying_cash_medi...,0.000000,0.000000,135
38,aggregate_travel_time_to_work,0.000000,0.000000,136


In [21]:
r2_train = las.score(c2_strain,Y_c2train)
r2_test = las.score(c2_stest,Y_c2test)
lasso_output['Clust_2'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA,Clust_2
RMSE_train,76.623570,76.576679,90.976706,92.078591,67.520566
RMSE_test,83.513629,86.460249,139.805073,79.462279,64.864915
R2_train,0.987961,0.987916,0.985649,0.985353,0.985127
R2_test,0.986911,0.986285,0.965790,0.987349,0.988706


In [22]:
Pkl_Filename = "Pickle_lasso_c2.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(las, file)

In [23]:
# Run a PCA on Cluster 2:
scaler_pca = StandardScaler().fit(c2_train)
features = scaler_pca.transform(c2_train)
df2_scal = pd.DataFrame(features, columns = c2_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df2_scal)

df_reduced = pd.DataFrame(pca.components_, columns = c2_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat2 = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat2.append(ff)

feat2.append('log_value')

c2_train = c2_train[feat2]
c2_test = c2_test[feat2]

# Run Lasso Regression on Cluster 2 with PCA variables:

scaler = StandardScaler().fit(c2_train)
features = scaler.transform(c2_train)
c2_strain = pd.DataFrame(features, columns = c2_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c2_strain, Y_c2train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c2_strain,Y_c2train)

print('Score for training data:', las.score(c2_strain,Y_c2train))

pred = las.predict(c2_strain)
pred = np.exp(pred)
Y_c2_train = np.exp(Y_c2train)
train_RMSE = mean_squared_error(Y_c2_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c2_test)
c2_stest = pd.DataFrame(test_features, columns = c2_test.columns)

print('Score for test data:', las.score(c2_stest,Y_c2test))

pred_test = las.predict(c2_stest)
pred_test = np.exp(pred_test)
Y_c2_test = np.exp(Y_c2test)
Test_RMSE = mean_squared_error(Y_c2_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c2_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9842203729648267
(not log) RMSE for training data is: 69.61442776333381
Score for test data: 0.9850445558698117
(not log) RMSE for test data is: 73.3776815721647


,Feature,Importance,Abs Importance,ranking
40,log_value,0.383393,0.383393,1
15,gdp,0.004409,0.004409,2
6,cpi,-0.002039,0.002039,3
3,median_income,0.001243,0.001243,4
19,employed_information,0.000666,0.000666,5
35,worked_at_home,0.000506,0.000506,6
18,dwellings_2_units,-0.000445,0.000445,7
26,housing_built_2000_to_2004,0.000081,0.000081,8
30,housing_built_1939_or_earlier,0.000000,0.000000,9
25,housing_built_2005_or_later,0.000000,0.000000,10


In [24]:
r2_train = las.score(c2_strain,Y_c2train)
r2_test = las.score(c2_stest,Y_c2test)
lasso_output['Clust_2_PCA'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA,Clust_2,Clust_2_PCA
RMSE_train,76.623570,76.576679,90.976706,92.078591,67.520566,69.614428
RMSE_test,83.513629,86.460249,139.805073,79.462279,64.864915,73.377682
R2_train,0.987961,0.987916,0.985649,0.985353,0.985127,0.984220
R2_test,0.986911,0.986285,0.965790,0.987349,0.988706,0.985045


In [25]:
Pkl_Filename = "Pickle_lasso_c2_PCA.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(las, file)

In [26]:
# Run Lasso Regression on Cluster 3:

scaler = StandardScaler().fit(c3_train)
features = scaler.transform(c3_train)
c3_strain = pd.DataFrame(features, columns = c3_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c3_strain, Y_c3train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c3_strain,Y_c3train)

print('Score for training data:', las.score(c3_strain,Y_c3train))

pred = las.predict(c3_strain)
pred = np.exp(pred)
Y_c3_train = np.exp(Y_c3train)
train_RMSE = mean_squared_error(Y_c3_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c3_test)
c3_stest = pd.DataFrame(test_features, columns = c3_test.columns)

print('Score for test data:', las.score(c3_stest,Y_c3test))

pred_test = las.predict(c3_stest)
pred_test = np.exp(pred_test)
Y_c3_test = np.exp(Y_c3test)
Test_RMSE = mean_squared_error(Y_c3_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c3_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.986681406202196
(not log) RMSE for training data is: 68.32932503577295
Score for test data: 0.9861354358606592
(not log) RMSE for test data is: 76.21227996320783


,Feature,Importance,Abs Importance,ranking
128,log_value,0.341305,0.341305,1
126,interest,-0.006401,0.006401,2
125,gdp,0.004590,0.004590,3
23,median_rent,0.003373,0.003373,4
40,owner_occupied_housing_units_lower_value_quartile,0.002996,0.002996,5
...,...,...,...,...
43,married_households,-0.000000,0.000000,133
42,owner_occupied_housing_units_upper_value_quartile,0.000000,0.000000,134
41,owner_occupied_housing_units_median_value,0.000000,0.000000,135
39,renter_occupied_housing_units_paying_cash_medi...,0.000000,0.000000,136


In [27]:
r2_train = las.score(c3_strain,Y_c3train)
r2_test = las.score(c3_stest,Y_c3test)
lasso_output['Clust_3'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA,Clust_2,Clust_2_PCA,Clust_3
RMSE_train,76.623570,76.576679,90.976706,92.078591,67.520566,69.614428,68.329325
RMSE_test,83.513629,86.460249,139.805073,79.462279,64.864915,73.377682,76.212280
R2_train,0.987961,0.987916,0.985649,0.985353,0.985127,0.984220,0.986681
R2_test,0.986911,0.986285,0.965790,0.987349,0.988706,0.985045,0.986135


In [28]:
Pkl_Filename = "Pickle_lasso_c3.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(las, file)

In [29]:
# Run a PCA on Cluster 3:
scaler_pca = StandardScaler().fit(c3_train)
features = scaler_pca.transform(c3_train)
df3_scal = pd.DataFrame(features, columns = c3_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df3_scal)

df_reduced = pd.DataFrame(pca.components_, columns = c3_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat3 = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat3.append(ff)

feat3.append('log_value')

c3_train = c3_train[feat3]
c3_test = c3_test[feat3]

# Run Lasso Regression on Cluster 3 with PCA variables:

scaler = StandardScaler().fit(c3_train)
features = scaler.transform(c3_train)
c3_strain = pd.DataFrame(features, columns = c3_train.columns)

alphas = 10**np.linspace(10,-2,500)*0.5

lassocv = LassoCV(alphas = alphas, random_state=0)
lassocv.fit(c3_strain, Y_c3train)
lassocv_alpha = lassocv.alpha_

las = Lasso(alpha = lassocv.alpha_, random_state = 0)
las.fit(c3_strain,Y_c3train)

print('Score for training data:', las.score(c3_strain,Y_c3train))

pred = las.predict(c3_strain)
pred = np.exp(pred)
Y_c3_train = np.exp(Y_c3train)
train_RMSE = mean_squared_error(Y_c3_train, pred, squared = False)
print('(not log) RMSE for training data is:', train_RMSE)

test_features = scaler.transform(c3_test)
c3_stest = pd.DataFrame(test_features, columns = c3_test.columns)

print('Score for test data:', las.score(c3_stest,Y_c3test))

pred_test = las.predict(c3_stest)
pred_test = np.exp(pred_test)
Y_c3_test = np.exp(Y_c3test)
Test_RMSE = mean_squared_error(Y_c3_test, pred_test, squared = False)
print('(not log) RMSE for test data is:', Test_RMSE)

las_imp = pd.DataFrame()

las_imp['Feature'] = c3_strain.columns
las_imp['Importance'] = list(las.coef_)
las_imp['Abs Importance'] = abs(las_imp['Importance'])
las_imp = las_imp.sort_values(by = 'Abs Importance', ascending = False)
las_imp

ranking = list(range(1,las_imp.shape[0]+1))
las_imp['ranking'] = ranking
las_imp

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Score for training data: 0.9856926121878472
(not log) RMSE for training data is: 72.04862292835256
Score for test data: 0.9819203672265758
(not log) RMSE for test data is: 89.34487955020683


,Feature,Importance,Abs Importance,ranking
34,log_value,0.345873,0.345873,1
15,gdp,0.005659,0.005659,2
6,cpi,-0.002378,0.002378,3
22,renter_occupied_housing_units_paying_cash_medi...,0.001101,0.001101,4
3,households_retirement_income,-0.000415,0.000415,5
21,commute_5_9_mins,-0.000077,0.000077,6
26,employed_education_health_social,-0.000000,0.000000,7
23,housing_built_2005_or_later,-0.000000,0.000000,8
24,commute_20_24_mins,-0.000000,0.000000,9
25,less_one_year_college,-0.000000,0.000000,10


In [30]:
r2_train = las.score(c3_strain,Y_c3train)
r2_test = las.score(c3_stest,Y_c3test)
lasso_output['Clust_3_PCA'] = [train_RMSE, Test_RMSE, r2_train, r2_test]
lasso_output

,all,all_PCA,Clust_1,Clust_1_PCA,Clust_2,Clust_2_PCA,Clust_3,Clust_3_PCA
RMSE_train,76.623570,76.576679,90.976706,92.078591,67.520566,69.614428,68.329325,72.048623
RMSE_test,83.513629,86.460249,139.805073,79.462279,64.864915,73.377682,76.212280,89.344880
R2_train,0.987961,0.987916,0.985649,0.985353,0.985127,0.984220,0.986681,0.985693
R2_test,0.986911,0.986285,0.965790,0.987349,0.988706,0.985045,0.986135,0.981920


In [31]:
Pkl_Filename = "Pickle_lasso_c3_PCA.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(las, file)